In [80]:
import requests
import keys
import pandas as pd
from datetime import datetime
import numpy as np
import json

In [81]:
# base_url = 'https://id.twitch.tv/oauth2/token'
# params = {
#     'client_id': keys.API_CLIENT_ID,
#     'client_secret': keys.API_CLIENT_SECRET,
#     'grant_type': 'client_credentials'
# }
# response = requests.post(base_url, params=params)


In [82]:
# print(response.json())

In [83]:
def fetch():
    base_url = 'https://api.igdb.com/v4/games'
    headers = {
        'Client-ID': keys.API_CLIENT_ID,
        'Authorization': f'Bearer {keys.API_AUTHORIZATION_TOKEN}',
    }
    all_data = []
    offset = 0

    while True:
        # Define the query for the game
        data = f"""fields id, slug, summary, 
     genres.slug, keywords.slug, name,
    first_release_date; 
    where rating > 70 & first_release_date>1111977600;
    limit 500; 
    offset {offset};"""

        try:
            # Make the POST request
            response = requests.post(base_url, headers=headers, data=data)

            # Check for non-200 status codes
            if response.status_code != 200:
                print(f"Error: {response.status_code}, {response.text}")
                return

            games = response.json()

            # Check if no data is returned
            if not games:
                print(f"No more games found at offset {offset}. Exiting.")
                break

            # Append the games to the list
            all_data.extend(games)
            print(f"Scraped {offset + len(games)} games")

            # Stop if fewer than 500 games are returned
            if len(games) < 500:
                break

            # Increment the offset
            offset += 500

        except requests.RequestException as e:
            print(f"Request failed: {e}")
            break

    return all_data

In [84]:
fetched=fetch()

Scraped 500 games
Scraped 1000 games
Scraped 1500 games
Scraped 2000 games
Scraped 2500 games
Scraped 3000 games
Scraped 3500 games
Scraped 4000 games
Scraped 4500 games
Scraped 5000 games
Scraped 5500 games
Scraped 6000 games
Scraped 6500 games
Scraped 7000 games
Scraped 7500 games
Scraped 8000 games
Scraped 8500 games
Scraped 8897 games


In [85]:
df=pd.DataFrame(fetched)

In [86]:
df.head()

,id,first_release_date,genres,keywords,name,slug,summary
0,6964,1173139200,"[{'id': 4, 'slug': 'fighting'}, {'id': 7, 'slu...","[{'id': 61, 'slug': 'martial-arts'}, {'id': 97...",Def Jam: Icon,def-jam-icon,"Infusing hip-hop music, culture and lifestyle ..."
1,11137,1521504000,"[{'id': 13, 'slug': 'simulator'}, {'id': 31, '...","[{'id': 4, 'slug': 'pirates'}, {'id': 72, 'slu...",Sea of Thieves,sea-of-thieves,Sea of Thieves offers the essential pirate exp...
2,122646,1563840000,"[{'id': 13, 'slug': 'simulator'}]","[{'id': 129, 'slug': 'bloody'}, {'id': 287, 's...",People Playground,people-playground,"Shoot, stab, burn, poison, tear, vaporise, or ..."
3,131999,1619049600,"[{'id': 12, 'slug': 'role-playing-rpg'}, {'id'...","[{'id': 1313, 'slug': 'multiple-endings'}, {'i...",NieR Replicant ver.1.22474487139...,nier-replicant-ver-dot-1-22474487139-dot-dot-dot,NieR Replicant ver.1.22474487139... is an upda...
4,394,1178582400,"[{'id': 12, 'slug': 'role-playing-rpg'}, {'id'...","[{'id': 78, 'slug': 'anime'}, {'id': 151, 'slu...",Final Fantasy Tactics: The War of the Lions,final-fantasy-tactics-the-war-of-the-lions,An updated version of the PlayStation game Fin...


In [87]:
def get_slug(items):
    if pd.isna(items).all() if isinstance(items, list) else pd.isna(items):
        return []
    
    return [
        item.get('slug', 'unknown')  
        for item in items
    ]



def get_time(unix):
    if pd.isna(unix):
        return None  
    return datetime.fromtimestamp(unix)


In [88]:
df['genres']=df['genres'].apply(get_slug)

In [89]:
df['keywords']=df['keywords'].apply(get_slug)

In [90]:
df['first_release_date']=df['first_release_date'].apply(get_time)

In [91]:
df.head()

,id,first_release_date,genres,keywords,name,slug,summary
0,6964,2007-03-06 05:30:00,"[fighting, music]","[martial-arts, hand-to-hand-combat, achievemen...",Def Jam: Icon,def-jam-icon,"Infusing hip-hop music, culture and lifestyle ..."
1,11137,2018-03-20 05:30:00,"[simulator, adventure]","[pirates, exploration, grinding, crafting, act...",Sea of Thieves,sea-of-thieves,Sea of Thieves offers the essential pirate exp...
2,122646,2019-07-23 05:30:00,[simulator],"[bloody, physics, 2d, death, gore, pixel-art, ...",People Playground,people-playground,"Shoot, stab, burn, poison, tear, vaporise, or ..."
3,131999,2021-04-22 05:30:00,"[role-playing-rpg, adventure]","[multiple-endings, apocalyptic]",NieR Replicant ver.1.22474487139...,nier-replicant-ver-dot-1-22474487139-dot-dot-dot,NieR Replicant ver.1.22474487139... is an upda...
4,394,2007-05-08 05:30:00,"[role-playing-rpg, simulator, strategy, turn-b...","[anime, medieval, jrpg, robots, party-based-co...",Final Fantasy Tactics: The War of the Lions,final-fantasy-tactics-the-war-of-the-lions,An updated version of the PlayStation game Fin...


In [92]:
df.shape

(8897, 7)

In [93]:
df.isnull().sum()

id                     0
first_release_date     0
genres                 0
keywords               0
name                   0
slug                   0
summary               24
dtype: int64

In [94]:
df.dropna(inplace=True)

In [95]:
df.to_csv('data.csv',index=False)